In [14]:
import os
import xlrd
import pprint
import sys
pp = pprint.PrettyPrinter()

In [15]:
sys.path.append('/home/simon/git/lipid_prototype')
sys.path.append('/home/simon/git/mass-spec-utils')
sys.path.append('/home/simon/git/pymzm')

In [16]:
excel_folder = '/home/simon/git/lipid_prototype/excel_input_files'
data_path = '/home/simon/data/lipid_files/lipid_files/Condition_1/Neg'

In [17]:
PERMITTED_HEADS = {'name','formula','ion',
                   'ion_mz (optional)',
                   'mz_tolerance_ppm (optional)',
                   'ion rt (seconds)',
                   'ion rt tol (seconds)',
                   'files to exclude'}

In [18]:
def read_heads(sheet_object,permitted_vals = PERMITTED_HEADS):
    col_pos = 0
    row_pos = 0
    head_dict = {}
    for col_pos in range(sheet_object.ncols):
        heads_val = sheet_object.cell_value(row_pos,col_pos)
        heads_val = heads_val.lower()
        if heads_val in permitted_vals:
            head_dict[heads_val] = col_pos
        else:
            print("Warning: head {} not permitted".format(heads_val))
    return head_dict

def load_lipids(sheet_object,permitted_vals = PERMITTED_HEADS):
    head_dict = read_heads(sheet_object,permitted_vals = permitted_vals)
    print(head_dict)
    lipids = {}
    for row_pos in range(sheet_object.nrows):
        if row_pos == 0:
            continue # skip the headings row
        lipid_name = sheet_object.cell_value(row_pos,head_dict['name'])
        lipids[lipid_name] = {}
        for key,col in head_dict.items():
            data_val = sheet_object.cell_value(row_pos,col)
            lipids[lipid_name][key] = data_val
        if not 'n_iso' in lipids[lipid_name]:
            lipids[lipid_name]['n_iso'] = 5
    return lipids
        
    
def load_files(sheet_object):
    assert sheet_object.cell_value(0,0).lower() == 'filename', print("First column in files sheet must be headed filename")
    assert sheet_object.cell_value(0,1).lower() == 'timepoint', print("Second column in files sheet must be headed timepoint")
    time_points = []
    for row_pos in range(1,sheet_object.nrows):
        file_name = sheet_object.cell_value(row_pos,0)
        timepoint = sheet_object.cell_value(row_pos,1)
        time_points.append((file_name,float(timepoint)))
    time_points.sort(key = lambda x: x[1])
    return time_points

def load_data(workbook_object,permitted_vals = PERMITTED_HEADS):
    lipid_sheet = workbook_object.sheet_by_name('lipids')
    lipids = load_lipids(lipid_sheet,permitted_vals = permitted_vals)
    files_sheet = workbook_object.sheet_by_name('files')
    time_points = load_files(files_sheet)
    return lipids,time_points

In [19]:
excel_file = os.path.join(excel_folder,'Test_input_Condition1Neg.xlsx')
wb = xlrd.open_workbook(excel_file) 

lipids,time_points = load_data(wb)
pp.pprint(lipids)
pp.pprint(time_points)

{'name': 0, 'formula': 1, 'ion': 2, 'ion_mz (optional)': 3, 'mz_tolerance_ppm (optional)': 4, 'ion rt (seconds)': 5, 'ion rt tol (seconds)': 6, 'files to exclude': 7}
{'38.1 pc': {'files to exclude': '',
             'formula': 'C46H90NO8P',
             'ion': '[M-H+FA]-',
             'ion rt (seconds)': 726.5999999999999,
             'ion rt tol (seconds)': 10.0,
             'ion_mz (optional)': 860.687,
             'mz_tolerance_ppm (optional)': 5.0,
             'n_iso': 5,
             'name': '38.1 pc'},
 '38.2 pc': {'files to exclude': '',
             'formula': 'C46H88NO8P',
             'ion': '[M-H+FA]-',
             'ion rt (seconds)': 627.6,
             'ion rt tol (seconds)': 10.0,
             'ion_mz (optional)': 858.6249,
             'mz_tolerance_ppm (optional)': 5.0,
             'n_iso': 5,
             'name': '38.2 pc'},
 '40.6 pc': {'files to exclude': '',
             'formula': 'C48H84NO8P',
             'ion': '[M-H]-',
             'ion rt (seconds)': 

In [20]:
mzml_file_objs = {}

from ms2_matching import MZMLFile
for filename,time in time_points:
    if not '.mzML' in filename:
        load_filename = filename + '.mzML'
    else:
        load_filename = filename
    full_file = os.path.join(data_path,load_filename)
    mzml_file_objs[filename] = MZMLFile(full_file)

/home/simon/.virtualenvs/lipid_prototype-tm1YvK8T/lib/python3.6/site-packages/pymzml/spec.py:458: FutureWarning: The behavior of this method will change in future versions.  Use specific 'len(elem)' or 'elem is not None' test instead.
  if self.element:
/home/simon/.virtualenvs/lipid_prototype-tm1YvK8T/lib/python3.6/site-packages/pymzml/spec.py:208: FutureWarning: The behavior of this method will change in future versions.  Use specific 'len(elem)' or 'elem is not None' test instead.
  if b_data_array:
/home/simon/.virtualenvs/lipid_prototype-tm1YvK8T/lib/python3.6/site-packages/pymzml/spec.py:473: FutureWarning: The behavior of this method will change in future versions.  Use specific 'len(elem)' or 'elem is not None' test instead.
  if self.element:


Loaded 1965 scans
Loaded 1965 scans
Loaded 1965 scans
Loaded 1965 scans
Loaded 1965 scans
Loaded 1965 scans
Loaded 1965 scans


In [21]:
%load_ext autoreload
%autoreload 2
from lipid_kinetics import compute_lipid_kinetics,create_plot
output_dict = {}
for lipid in lipids:
#     if lipid in output_dict:
#         continue
    print(lipid,lipids[lipid]['ion'])
    rt_mean = lipids[lipid]['ion rt (seconds)']
    rt_tol = lipids[lipid]['ion rt tol (seconds)']
    lipids[lipid]['rt_range'] = [rt_mean - rt_tol,rt_mean+rt_tol]
    lipids[lipid]['adduct_type'] = lipids[lipid]['ion']
    output_dict[lipid] = compute_lipid_kinetics(lipid,lipids[lipid],time_points,mzml_file_objs)
    create_plot(lipid,output_dict[lipid])




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Pe 37.1 [M-H]-
pc 34:2 [M-H+FA]-
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv fi

/home/simon/.virtualenvs/lipid_prototype-tm1YvK8T/lib/python3.6/site-packages/pymzml/spec.py:473: FutureWarning: The behavior of this method will change in future versions.  Use specific 'len(elem)' or 'elem is not None' test instead.
  if self.element:


PC(36:2) [M-H+FA]-
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv fil

/home/simon/git/lipid_prototype/lipid_kinetics.py:179: RuntimeWarning: invalid value encountered in true_divide
  data_mat/=data_mat.sum(axis=1)[:,None]


PC(34:0) [M-H+FA]-
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv fil

...

/home/simon/git/lipid_prototype/lipid_kinetics.py:179: RuntimeWarning: invalid value encountered in true_divide
  data_mat/=data_mat.sum(axis=1)[:,None]


PC 32:0 [M-H+FA]-
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv file

/home/simon/git/lipid_prototype/lipid_kinetics.py:179: RuntimeWarning: invalid value encountered in true_divide
  data_mat/=data_mat.sum(axis=1)[:,None]
/home/simon/git/lipid_prototype/lipid_kinetics.py:179: RuntimeWarning: invalid value encountered in true_divide
  data_mat/=data_mat.sum(axis=1)[:,None]


PC(38:4p) [M-H+FA]-
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv files. Assuming normal dialect.
[M-H+CH2O2]- not a valid adduct, trying to parse
Failed to load .csv fi

/home/simon/git/lipid_prototype/lipid_kinetics.py:179: RuntimeWarning: invalid value encountered in true_divide
  data_mat/=data_mat.sum(axis=1)[:,None]


In [22]:
from lipid_kinetics import create_xlsx_output
xlsx_output_name = 'condition_1_neg.xlsx'
create_xlsx_output(output_dict,output_filename = xlsx_output_name)

Writing:  temp_0.png
Writing:  temp_1.png
Writing:  temp_2.png
Writing:  temp_3.png
Writing:  temp_4.png
Writing:  temp_5.png
Writing:  temp_6.png
Writing:  temp_7.png
Writing:  temp_8.png
Writing:  temp_9.png
Writing:  temp_10.png
Writing:  temp_11.png
Writing:  temp_12.png
Writing:  temp_13.png
Writing:  temp_14.png
Writing:  temp_15.png
Writing:  temp_16.png
Writing:  temp_17.png
Writing:  temp_18.png
Writing:  temp_19.png
Writing:  temp_20.png
Writing:  temp_21.png
Writing:  temp_22.png
Writing:  temp_23.png
Writing:  temp_24.png
Writing:  temp_25.png
Writing:  temp_26.png
Writing:  temp_27.png
Writing:  temp_28.png
Writing:  temp_29.png
Writing:  temp_30.png
Writing:  temp_31.png
Writing:  temp_32.png
Writing:  temp_33.png
Writing:  temp_34.png
Writing:  temp_35.png
Writing:  temp_36.png
Writing:  temp_37.png
